# Data generation

First go at the data generation script. Script taken and adpated from matlab script for master thesis (IV estimation).

Update as needed

In [8]:
import numpy as np
import itertools
import math

In [19]:
n = 100        # n observations
m = 5          # n instruments
rho = 0.6      # Corr(eps, U)
theta0 = 1     # true structural coefficient 
density = 0.4  # signal density
CP = 150       # concentration parameter

In [36]:
# Var-cov matrix of instruments
V_z = np.zeros((m, m))
vec_ix = list(itertools.product(range(m), range(m)))
for i, j in vec_ix:
    V_z[i, j] = (math.sqrt(0.3) ** 2) * (0.5 ** abs(i - j))

# Error covariance ----------------------------------------
pi = np.zeros(math.ceil(m))
pi[:math.ceil(m * density)]  = 1.0                    # First stage coefficients
V_U = n * np.matmul(np.matmul(pi.T, V_z), pi) / CP    # Variance of first stage error
V_eps = 2                                             # Variance of second stage error
V_eps_U = rho * np.sqrt(V_eps) * np.sqrt(V_U)         # Cov(eps, U) (endogeneity)
V_err = np.array([[V_eps, V_eps_U], [V_eps_U, V_U]])  # Var(eps, U)

In [63]:
def x_fn(z, u, pi):
    return np.matmul(z, pi) + u

def y_fn(x, e, theta):
    return x.T * theta + e

In [49]:
np.random.seed(seed=1)

E = np.matmul(np.random.rand(n, 2), np.linalg.cholesky(V_err))
e = E[:, 0]
u = E[:, 1]
np.mean(e)

0.8774843696213764

In [64]:
np.random.seed(seed=1)

E = np.matmul(np.random.rand(n, 2), np.linalg.cholesky(V_err))
e = E[:, 0]
u = E[:, 1]
e = e - np.mean(e)
u = u - np.mean(u)

z = np.matmul(np.random.rand(n, m), np.linalg.cholesky(V_z))
z = z - np.mean(z)

x = x_fn(z, u, pi)

y = y_fn(x, e, theta0)